# Table of contents:

* [Introduction to block ciphers](#intro-block)
* [Padding a message](#message-padding)
* [The Advanced Encryption Standard (AES)](#AES)
* [Modes of operation of block ciphers](#modes)
* [Size of the output ciphertex on AES](#size)
* [Bonus: Fernet cipher](#fernet)
    
Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.



## Block Ciphers <a class="anchor" id="intro-block"></a>

Block ciphers as opposed to stream ciphers take a block of the plaintext (a specific amount of bytes) and encrypts it into a block with the same size. In this section we will use the Advanced Encryption Standard (AES) to understand block ciphers. In the next schema it is shown how an original message of arbitrary $N$ bytes is converted into a ciphertext having blocks of $K$ bytes. The ciphertext size is always a multiple of $K$ bytes.

<img src="img/block_cipher.png" style="width:1100px"/>

## Padding a message <a class="anchor" id="message-padding"></a>

Most of the times the lenght of the message is not a multiple of the block size so we need to "pad" the message to have the required length. A common padding function is [PKCS7](https://en.wikipedia.org/wiki/Padding_(cryptography)). Basically what PKCS7 does is appendinng a list of bytes with the same value corresponding to the number of bytes needed to complete the block.




In [1]:
#Padding Using PKCS7
def pkcs7padding(input, block_size=16):
  if (type(input) != bytearray and type(input) != bytes):
    raise TypeError("Only support byte array")
    
  padding = block_size - (len(input) % block_size)
  return input + bytearray([padding for i in range(padding)])

def pkcs5padding(input):
    return pkcs7padding(input, 8)

## Encrypting using AES (Advanced Encryption Standard) <a class="anchor" id="AES"></a>

AES is a block cipher that was established as a standard by NIST in 2001 (after a public call to improve/substitute DES encryption algorithm in 1997). AES is a subset of the Rijndael block cipher developed by Vincent Rijmen and Joan Daemen submitted to NIST during the [AES selection process](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard_process).


We are not going to go into the details of te exact implementation but the readers are referred to the book of [Katz and Lindell](http://www.cs.umd.edu/~jkatz/imc.html) Chapter 6 section 2. Also Mike Pound explains AES in this [video](https://www.youtube.com/watch?v=O4xNJsjtN6E&t=524s&ab_channel=Computerphile), check it out!

In [2]:
#Importing and installing libraries
!pip install pycryptodomex
from Cryptodome.Cipher import AES

#Creating a private key and encrypting text using AES
PrivateKey = b'raneemahmed12345'
cipher = AES.new(PrivateKey, AES.MODE_EAX)
Text_to_be_encrypted = b'City under siege'
nonce = cipher.nonce
cipheredText, tag = cipher.encrypt_and_digest(Text_to_be_encrypted)
print("PlainText is: ",Text_to_be_encrypted ,"Ciphertext is: ", cipheredText)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
PlainText is:  b'City under siege' Ciphertext is:  b'<\x9d}\x96\xecUDD\x1cgl\xa0\x9b\x83\xb5\xbb'


## Modes of operation of block ciphers <a class="anchor" id="mode"></a>

A block cipher by itself is only suitable for the secure cryptographic transformation (encryption or decryption) of one fixed-length group of bits called a block. A mode of operation describes how to repeatedly apply a cipher's single-block operation to securely transform amounts of data larger than a block ([Wikipedia](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation)).

The first mode is "not doing anything", this is the Electronic Codebook mode. See the figure below (from Wikipedia).

<img src="img/ECB_mode.png" style="width:1100px"/>

We are lucky and in ```cryptography``` package ECB implemented in ```cryptography.hazmat.primitives.ciphers.ECB``` function (we've seen in the previous example!).

Now we can encrypt the same message twice and see what we get in the ciphertext:

This is not a desirable outcome. If I want to send the same message twice, I really don't want to send the same ciphertext. What if in all comunications I start by "Dear..." and the attacker knows it?. A better mode is the Cipher block chaining (CBC):

<img src="img/CBC_mode.png" style="width:1100px"/>

In this case we take a random initialization vector and perform XOR operation with the block of plaintext, then we feed this into the encryptor, after that we obtain the ciphertext. This ciphertext is used as the initialization vector to encrypt the next block.

# **Electronic Codebook encryption (ECB) :**

In [5]:
#Installing and importing libraries
!pip install pycryptodomex
from base64 import b64encode
from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes

#Encryption using ECB
Text_to_be_encrypted = b"Encrypting w ECB"
PrivateKey = get_random_bytes(16)

cipher = AES.new(PrivateKey, AES.MODE_ECB)
cipheredText = cipher.encrypt(Text_to_be_encrypted)

print('Plain Text:',Text_to_be_encrypted, ' Cipher text:',cipheredText)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Plain Text: b'Encrypting w ECB'  Cipher text: b'\xc2\xf6=/\xf0\xee\x0b\x1b\xb8\x1aF\xa5\x0e!|\\'


# **Cipher Block chaining encryption (CBC):**

In [4]:
#Installing and importing packages and libraries
!pip install pycryptodomex
from base64 import b64encode
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad
from Cryptodome.Random import get_random_bytes

#Encryption Using CBC
Text_to_be_encrypted = b"Encrypting w cbc"
PrivateKey = get_random_bytes(16)

cipher = AES.new(PrivateKey, AES.MODE_CBC)
cipher_bytes = cipher.encrypt(pad(Text_to_be_encrypted, AES.block_size))

iv = b64encode(cipher.iv).decode('utf-8')
cipheredText = b64encode(cipher_bytes).decode('utf-8')

print('Plain Text',Text_to_be_encrypted,' IV:',iv, ' Cipher text:',cipheredText)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Plain Text b'Encrypting w cbc'  IV: e3ckFhHea8bkOHJCzFiZHA==  Cipher text: kxrj5MatfWOxoIJKrrYj/WjOFbHgU3yXLn7CHrPYguA=


## Size of ciphertext <a class="anchor" id="size"></a>

## Bonus: Fernet <a class="anchor" id="fernet"></a>

Another block cipher implemented in cryptography package is [Fernet](https://asecuritysite.com/encryption/fernet). 